#Projet introduction à l'analyse d'image

In [ ]:
!pip install pillow-heif
from google.colab import drive
import os
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pillow_heif
import csv
import math
from functools import cmp_to_key
from google.colab.patches import cv2_imshow

drive.mount('/content/gdrive')


## Fonctions pour le prétraitement des images

In [ ]:
# Resize an image without distortion using OpenCV
# Source: https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
def image_resize(image, width=None, height=None, inter=cv.INTER_AREA):
    # Initialize the dimensions of the image to be resized and grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # If both the width and height are None, then return the original image
    if width is None and height is None:
        return image

    # Check to see if the width is None
    elif width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # Check to see if the height is None
    elif height is None:
        # Calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Otherwise, take the least impacting parameter
    else:
        # Calculate the ratios
        r_w = width / float(w)
        r_h = height / float(h)

        # Calculate the least affected dimensions (good for sideway photos)
        if r_w <= r_h:
            dim = (int(w * r_h), height)
        else:
            dim = (width, int(h * r_w))


    # Resize the image
    resized = cv.resize(image, dim, interpolation=inter)

    # Return the resized image
    return resized

################################################################################

# Function to extract regions of interest (ROIs) using detected circles
def extract_ROIs(circles, image):
    ROIs = []

    for i in circles[0, :]:
        # Prepare a black canvas
        height, width = image.shape[:2]
        canvas = np.zeros((height, width))

        # Draw the outer circle
        color = (255, 255, 255)
        thickness = -1
        centerX = i[0]
        centerY = i[1]
        radius = i[2]

        # If the circle's radius is too small, skip it
        if radius < 9:
            continue

        cv.circle(canvas, (centerX, centerY), radius, color, thickness)

        # Create a copy of the input and mask input
        image_copy = image.copy()
        image_copy[canvas == 0] = (0, 0, 0)

        # Crop the ROI
        x = centerX - radius
        y = centerY - radius
        h = 2 * radius
        w = 2 * radius
        cropped_img = image_copy[y - 3:y + h + 3, x - 3:x + w + 3]

        # Store the ROI
        ROIs.append(cropped_img)
    return ROIs

################################################################################

# Functions for categorizing cent coins

RED_CENT_VALUE = (0.01 + 0.02 + 0.05) / 3.
YELLOW_CENT_VALUE = (0.10 + 0.20 + 0.50) / 3.

def is_red_cent(image, mask):
    # Check if the coin is a red cent based on its color properties
    mean_color = cv.mean(image, mask=mask)[:3]
    prc_GR = mean_color[1] / mean_color[2]
    prc_BR = mean_color[0] / mean_color[2]
    return prc_GR > prc_BR and (prc_GR - prc_BR) < .45

def is_yellow_cent(image, mask):
    # Check if the coin is a yellow cent based on its color properties
    mean_color = cv.mean(image, mask=mask)[:3]
    diff_RG = mean_color[2] - mean_color[1]
    diff_GB = mean_color[1] - mean_color[0]
    return diff_GB > diff_RG * 1.25

def get_cent_value(image):
    # Determine the value of the coin based on its color properties
    coin_mask = np.zeros(image.shape[:2], np.uint8)
    center = (49, 49)
    radius = 42
    cv.circle(coin_mask, center, radius, 255, -1)

    if is_yellow_cent(image, coin_mask):
        return YELLOW_CENT_VALUE
    elif is_red_cent(image, coin_mask):
        return RED_CENT_VALUE
    else:
        return 0.00

################################################################################

# Function to fix circle size if it protrudes the image boundaries
def fix_circle_size(circle, center, radius, image):
    height, width = image.shape[:2]
    height -= 1
    width -= 1

    center_x = float(center[0])
    center_y = float(center[1])

    # Check if the circle is protruding beyond the image boundaries
    if (center_y + radius >= height) or \
       (center_y - radius <= 0) or \
       (center_x + radius >= width) or \
       (center_x - radius <= 0):

        new_radius = np.uint16(max(min(height - center_y - 5, center_y - 5, \
                                   width - center_x - 5, center_x - 5), 0))
    else:
        new_radius = radius

    # Ensure the new radius does not cause overflow
    if new_radius < radius:
        circle[2] = new_radius

    return int(new_radius)


## Fonction permettant la lecture des images

In [ ]:
def open_image(image_name, folder_name):
    image = cv.imread(f"{folder_name}/{image_name}")

    # Try opening the image and convert .HEIC format to png
    if ".HEIC" in image_name:
        if os.path.exists(f"{folder_name}/{image_name}.png"):
            try:
                os.remove(f"{folder_name}/{image_name}")
            except:
                image = cv.imread(f"{folder_name}/{image_name}.png")
        else:
            try:
                heif_file = pillow_heif.open_heif(f"{folder_name}/{image_name}", \
                                                  convert_hdr_to_8bit=False, bgr_mode=True)
                np_array = np.asarray(heif_file)
                cv.imwrite(f"{folder_name}/{image_name}.png", np_array)
                image = cv.imread(f"{folder_name}/{image_name}.png")
            except:
                print('Could not open or find the image named:', image_name)
        print("Successfully opened the following .heic image:", image_name)
    elif image is not None:
        print("Successfully opened the following image:", image_name)
    elif image is None:
        print('Could not open or find the image named:', image_name)
        return

    return image



## Importation des images et de leurs info
(à changer en fonction de l'emplacement dans le drive)

In [ ]:
import pandas as pd

folder_name = "/content/gdrive/MyDrive/ColabNotebooks_image/validation_dataset"

df = pd.read_csv("/content/gdrive/MyDrive/ColabNotebooks_image/validation_dataset/data_annotations.csv")
df

## Fonction pour la détection du nombre de pieces dans les images

In [ ]:



# Detects coins on a given image and returns an array of OpenCV Circle objects
# and the total count of detected coins as an integer
def detect_coins(image):

    image_copy = image.copy()

    gray_image = cv.cvtColor(image_copy, cv.COLOR_BGR2GRAY)

    # Prepare the image by applying median blur and gaussian blur
    blur_image = cv.medianBlur(gray_image, 5)
    blur_image = cv.GaussianBlur(blur_image, (1,1), 0)

    # Then apply the Canny edge detection algorithm
    canny_image = cv.Canny(blur_image,75,200)

    image_for_hough_transform = canny_image

    # Hough Transform function
    circles = cv.HoughCircles(image_for_hough_transform, cv.HOUGH_GRADIENT, 1, \
                            minDist = 15, \
                            param1 = 120, param2 = 20, \
                            minRadius = 10, maxRadius = 150)
    # minDist: Minimum distance between the centers of the detected circles.
    # param1: The larger of the two thresholds passed to the Canny edge detector.
    # param2: Threshold for the minimum number of votes required for a circle's
    #         center to be detected. A smaller value may lead to false detections.
    # minRadius: The minimum radius of the circles to detect.
    # maxRadius: The maximum radius of the circles to detect.

    coin_count = 0

    # Iterate over detected circles
    if circles is not None and len(circles) > 0:

        circles = np.uint16(np.around(circles))

        # Sort the radiuses to find the median value
        sorted_radiuses = sorted(circles[0,:,2])
        median_radius = sorted_radiuses[len(sorted_radiuses)//2]

        # Selecting new Hough Transform parameters based on the median radius
        upper_radius_limit = round(median_radius * 1.4)
        lower_radius_limit = sorted_radiuses[0]
        lower_distance_limit = round(median_radius * .4) * 2

        # Second Hough Transform with the new parameters
        circles = cv.HoughCircles(image_for_hough_transform, cv.HOUGH_GRADIENT, 1, \
                                    minDist = lower_distance_limit, \
                                    param1 = 170, param2 = 20, \
                                    minRadius = lower_radius_limit, maxRadius = upper_radius_limit)
        # minDist: Minimum distance between the centers of the detected circles.
        # param1: The larger of the two thresholds passed to the Canny edge detector.
        # param2: Threshold for the minimum number of votes required for a circle's
        #         center to be detected. A smaller value may lead to false detections.
        # minRadius: The minimum radius of the circles to detect.
        # maxRadius: The maximum radius of the circles to detect.

        # Iterate over detected circles
        if circles is not None and len(circles) > 0:
            circles = np.uint16(np.around(circles))

            circle_list = circles[0,:]
            for circle_index in range(len(circle_list)):
                circle = circle_list[circle_index]

                # Extract circle parameters
                center = (circle[0], circle[1])  # Center coordinates
                radius = circle[2]  # Radius

                # If a circle is protruding the image boundaries, reduce its radius to fit
                radius = fix_circle_size(circle, center, radius, image_copy)

                # Draw the outer circle
                cv.circle(image_copy, center, radius, (0, 191, 255), 2)
                # Draw the center point
                cv.circle(image_copy, center, 2, (255, 255, 0), 2)

                coin_count += 1

        # Filtering circles by considering circles of radius < 9 as noise
        circles_temp = []
        for circle in circles[0,:]:
            radius = circle[2]
            if radius > 9:
                circles_temp.append(circle)

        circles = np.array([circles_temp])
        cv.circle(image_copy, (circles[0,0,0], circles[0,0,1]), \
                  circles[0,0,2], (0, 191, 255), 2)

        """
        # Draw the total count on the image
        fontType = cv.FONT_HERSHEY_DUPLEX
        cv.putText(image_copy, f'{str(coin_count)} coins', (30,30), fontType, \
                1, (255, 255, 0), 1, cv.LINE_AA)
        cv2_imshow(image_copy)
        """

        return circles, coin_count



## Fonction pour la detection de valeur pour chaque pieces détecté

In [ ]:
# Detects the total sum of coins in a provided image.
# Returns the sum in euros as a float.
def detect_total_coin_value(circles, original_image, resized_image=None):

    # If the image was resized, the circles are scaled back to the original image's scale
    if resized_image is not None:
        original_height = original_image.shape[0]
        resized_height = resized_image.shape[0]
        resize_ratio = original_height / resized_height
        circles[0] = circles[0,:] * resize_ratio

    total_value = 0.00

    # Extract the regions of interest of the image (circles) and isolate them into
    # individual images
    ROIs = extract_ROIs(circles, original_image)

    # Detect the value of each coin
    for ROI in ROIs:
        coin_value = 0.00
        circles = None # Reset the circles variable (optional)

        # Uniformize the isolated coin images
        ROI = image_resize(ROI, width = 100, height = 100)

        ROI_copy = ROI.copy()

        ROI_gray = cv.cvtColor(ROI_copy, cv.COLOR_BGR2GRAY)

        ROI_blur = cv.medianBlur(ROI_gray, 5)
        ROI_blur = cv.GaussianBlur(ROI_blur, (3,3), 0)

        # Hough Transform function to detect potential rings inside the coin
        circles = cv.HoughCircles(ROI_blur, cv.HOUGH_GRADIENT, \
                                1, minDist = 2, \
                                param1 = 120, param2 = 1, \
                                minRadius = 28, maxRadius = 35)
        # minDist: Minimum distance between the centers of the detected circles.
        # param1: The larger of the two thresholds passed to the Canny edge detector.
        # param2: Threshold for the minimum number of votes required for a circle's
        #         center to be detected. A smaller value may lead to false detections.
        # minRadius: The minimum radius of the circles to detect.
        # maxRadius: The maximum radius of the circles to detect.

        if circles is not None and len(circles) > 0:
            circles = np.uint16(np.around(circles))
            center_point = np.array([49, 49], dtype=np.uint16)
            points = circles[0,:]

            # Sort the circles by their distance to the center of the image
            points = np.array(sorted(points, \
                                    key=lambda item: \
                                    cv.norm(np.int8(item[:2]), np.int8(center_point[:2]))))

            # If the most centered detected ring is still significantly off-centered,
            # ignore it: it is not a euro coin
            if cv.norm(points[0,:2], center_point) > 9:
                total_value += get_cent_value(ROI)
                continue

            # Otherwise, consider the possibility of it being the inner ring of
            # a euro coin
            inner_circle = points[0]

            cv.circle(ROI_copy,(inner_circle[0], inner_circle[1]), inner_circle[2], (0, 191, 255), 2)
            cv.circle(ROI_copy,(inner_circle[0], inner_circle[1]), 2, (255, 255, 0), 2)

            inner_center = np.array(inner_circle[:2])
            inner_radius = inner_circle[2]

            # Calculate area of the inner circle
            inner_area = np.pi * inner_radius ** 2

            # Create a mask for this inner region
            inner_mask = np.zeros(ROI_copy.shape[:2], np.uint8)
            cv.circle(inner_mask, inner_center, inner_radius, 255, -1)

            # Calculate the mean color within the circular region
            ROI_copy = ROI.copy()
            inner_mean_color = cv.mean(ROI_copy, mask=inner_mask)

            # Second Hough Transform to detect the outer circle to check if it is a euro coin, or not
            circles = cv.HoughCircles(ROI_blur, cv.HOUGH_GRADIENT, 1, 5, \
                                    param1 = 120, param2 = 5, \
                                    minRadius = 37, maxRadius = 55)
            # minDist: Minimum distance between the centers of the detected circles.
            # param1: The larger of the two thresholds passed to the Canny edge detector.
            # param2: Threshold for the minimum number of votes required for a circle's
            #         center to be detected. A smaller value may lead to false detections.
            # minRadius: The minimum radius of the circles to detect.
            # maxRadius: The maximum radius of the circles to detect.

            if circles is not None and len(circles) > 0:

                points = circles[0,:]
                points = np.uint16(np.around(points))

                # Sort the outer circles by their distance to the center of the image
                points = sorted(points, \
                                key=lambda item: \
                                cv.norm(np.int8(item[:2]), np.int8(center_point[:2])))

                # For each detected outer circle, check if it is meaningful for
                # the euro coin validation task
                for outer_circle in points:

                    # Reduce its radius by a small amount to offset any margin between
                    # the image and the coin
                    if outer_circle[2] > 12:
                        outer_circle[2] -= 3

                    # If the current outer ring is too off-centered, the next ones
                    # will be worse for the evaluation of the coin: not a euro coin
                    if cv.norm(outer_circle[:2], center_point) > 10:
                        break

                    ROI_copy = ROI.copy()

                    cv.circle(ROI_copy,(outer_circle[0], outer_circle[1]), outer_circle[2], (0, 191, 255), 2)
                    cv.circle(ROI_copy,(outer_circle[0], outer_circle[1]), 2, (255, 255, 0), 2)

                    outer_center = np.array(outer_circle[:2])

                    # If the two circles have distant center points, we cannot consider
                    # it for the euro coin evaluation
                    if cv.norm(inner_center, outer_center) > 6:
                        continue

                    # If the radius of the larger ring is too similar to the inner ring,
                    # we cannot consider it for the euro coin evaluation
                    outer_radius = outer_circle[2]
                    if outer_radius < inner_radius * 1.2:
                        continue

                    # Calculate area of the outer ring
                    outer_area = (np.pi * outer_radius ** 2) - inner_area

                    # Create a mask for the outer ring
                    outer_mask = np.zeros(ROI_copy.shape[:2], np.uint8)
                    cv.circle(outer_mask, outer_center, outer_radius, 255, -1)
                    cv.circle(outer_mask, outer_center, inner_radius, 0, -1)

                    ROI_copy = ROI.copy()
                    cv.bitwise_and(ROI_copy,ROI_copy,mask = outer_mask)

                    # Calculate the mean color within this ring
                    outer_mean_color = cv.mean(ROI_copy, mask=outer_mask)

                    # If the inner circle is yellower than the outer ring, and
                    # that the difference is important enough, it is a 2 euro coin
                    if (((inner_mean_color[1] - inner_mean_color[0]) + \
                        (inner_mean_color[2] - inner_mean_color[0])) > \
                        ((outer_mean_color[1] - outer_mean_color[0]) + \
                        (outer_mean_color[2] - outer_mean_color[0]))) and \
                        cv.norm(inner_mean_color, outer_mean_color) > 20:

                        coin_value = 2.00
                        break

                    # If it is the opposite case, but also with a significant
                    # difference in colors, it is a 1 euro coin
                    elif cv.norm(inner_mean_color, outer_mean_color) > 20:
                        coin_value = 1.00
                        break

                    # Otherwise, it is not a euro coin
                    else:
                        coin_value = 0.00
                        break

                # If the coin was not detected to be a euro coin, try to find its
                # value in cents
                if coin_value == 0.00:
                    coin_value = get_cent_value(ROI)

            # If the coin was not detected to be a euro coin, try to find its
            # value in cents
            else:
                coin_value = get_cent_value(ROI)

        # If the coin was not detected to be a euro coin, try to find its
        # value in cents
        else:
            coin_value = get_cent_value(ROI)

        total_value += coin_value

    return total_value


### Test pour une image

In [ ]:
image = open_image("014.jpeg", folder_name)

# Scaling the image down to make it easier to detect the coins and to make
# the algorithms work faster
image_copy = image.copy()
resized_image = image_resize(image_copy, width = 400, height = 400)

print("ETAPE 1")
circles, coins = detect_coins(resized_image)
print(f"Nombre de pièces detectées : {coins}")

print("\nETAPE 2")
value = detect_total_coin_value(circles, image, resized_image)
print(f"Valeur totale : {round(value, 2)}")

## Fonction pour obtenir la valeur des pieces d'une image grace au csv

In [ ]:
def convert_value_to_numeric(value_str):
    words = value_str.split()
    euros = 0
    centimes = 0

    for i in range(len(words)):
        if words[i] == 'euros' or words[i] == 'euro':
            euros = int(words[i-1]) if i > 0 else 0
        elif words[i] == 'centimes' or words[i] == 'centime':
            centimes = int(words[i-1]) if i > 0 else 0

    total_value = euros + centimes / 100
    return total_value

## Calcul de la MAE pour les images detectées avec le bon nombre de pieces

In [ ]:
from sklearn.metrics import mean_absolute_error

print(df)

real_values = []
predicted_values = []

for index, row in df.iterrows():
    filename = row['image_name']
    num_real_pieces = row['nb_piece']
    num_real_value = row['value ']
    num_real_value_numeric = convert_value_to_numeric(num_real_value)

    print(f"IMAGE {filename} :")

    image = open_image(filename, folder_name)
    cirles, coins = detect_coins(image)
    print(f"cercles detecté : {coins}")
    print(f"nb piece reel : {num_real_pieces}")
    if(coins==num_real_pieces):
      value = detect_total_coin_value(circles, image)
      print(f"valeur detecté : {value}")
      print(f"valeur reel : {num_real_value_numeric}\n")
      real_values.append(num_real_value_numeric)
      predicted_values.append(value)
    else:
      print("Mauvais compte de piece")

mae = mean_absolute_error(real_values, predicted_values)
print(f"Mean Absolute Error : {mae}")

## Calcul du pourcentage d'images détectées avec le bon nombre de pieces

In [ ]:
total_images = 0
correct_images = 0

for index, row in df.iterrows():
    filename = row['image_name']
    num_real_pieces = row['nb_piece']
    num_real_value = row['value']

    total_images += 1

    print(f"IMAGE {filename} :")

    image = open_image(filename, folder_name)
    circles, coins = detect_coins(image)
    print(f"cercles détectés : {coins}")
    print(f"nb pièces réelles : {num_real_pieces}")

    if coins == num_real_pieces:
        correct_images += 1

percentage_correct = (correct_images / total_images) * 100
print(f"Pourcentage d'images avec le bon nombre de pièces : {percentage_correct}%")
